In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import chromedriver_autoinstaller
import os
import time
from selenium.webdriver.common.by import By

df = pd.read_csv('./루리웹__게임 리스트.csv')
review_list = df.iloc[:,[2]].values.tolist()


review_data = pd.DataFrame({'게임명':[], "리뷰":[], "추천":[], "비추천":[], "조회":[], "날짜":[]})


chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')   
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')

driver.implicitly_wait(10)

<ipython-input-1-083c3e1b2cab>:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')


In [3]:
df.head()

,Unnamed: 0,Name(Eng),Name(Kor),Release date
0,NaN,Rain World,레인 월드,2017-03-27
1,NaN,Kero Blaster,케로 블래스터,2014-05-11
2,NaN,Valkyrie Drive: Bhikkhuni,발키리 드라이브 비쿠니,2015-12-10
3,NaN,Headmaster,헤드마스터,2016-10-13
4,NaN,NEON STRUCT,네온 스트럭트,2015-04-03


In [ ]:
def reviewtext():
    for nums in range(1, 32):   # 후기 게시글 텍스트화하는 코드
        driver.find_element("xpath",'//*[@id="board_list"]/div/div[2]/table/tbody/tr[%d]/td[3]/div/a[1]' % (nums)).click()
        driver.find_element("xpath",'//*[@id="board_read"]/div/div[2]/div[2]/div[1]').text # 메인 글
        driver.find_element("xpath",'//*[@id="board_read"]/div/div[2]/div[1]/div[1]/div[2]/div[2]/div/p[5]').text # 추천, 조회수, 비추천
        driver.find_element("xpath",'//*[@id="board_read"]/div/div[2]/div[1]/div[1]/div[2]/div[2]/div/p[6]/span').text # 날짜

In [22]:
iwantreview = ["후기", "리뷰"]
reviwe_list = []

num = 0
for review in review_list:
    review = review[0]
    print('{}의 게임 리뷰를 수집합니다.'.format(review))
    # 루리웹 게임찾기 페이지
    driver.implicitly_wait(1)
    driver.get('https://bbs.ruliweb.com/game')
    time.sleep(2)
    # 게임 리스트 검색
    driver.find_element("xpath",'//*[@id="game_search"]/form/div[1]/div[2]/div/div[4]/div[1]/div/input[5]').send_keys(review)
    driver.find_element("xpath",'//*[@id="game_search"]/form/div[1]/div[2]/div/div[4]/div[1]/div/button').click()
    time.sleep(1)
    
    if driver.find_element("xpath", '//*[@id="game_search"]/form/ul/li/div/div[2]/div/p') == True: # 검색 결과없음 창이 뜬다면?
        print('게임 데이터가 없습니다.')
    else:
        try: # 에러가 발생할 수 있는 부분
            driver.find_element("xpath", '//*[@id="game_search"]/form/ul/li/div/div[2]/div/ul/a/li[1]/strong').click()
            time.sleep(1)
            # '후기' 검색
            driver.find_element("xpath",'//*[@id="board_list"]/div/div[3]/div[3]/div/input').send_keys(iwantreview[0]) # 후기 검색
            driver.find_element("xpath",'//*[@id="board_list"]/div/div[3]/div[3]/div/button').click()
            time.sleep(1)
            # 후기 리스트 데이터 프레임에 집어넣는 코드
            if driver.find_element("xpath",'//*[@id="board_list"]/div/div[2]/table/tbody/tr[2]/td/p') == True: # 만약 게시글이 없을 경우 패스
                print('후기 게시물이 없습니다.')
                driver.back()
            else:
                driver.find_element("xpath",'//*[@id="board_list"]/div/div[2]/table/thead/tr/th[5]/a').click() # 후기 게시글이 있을 경우 추천수 내림차순
                print('후기를 스크랩하고 있습니다.')
                time.sleep(1)
                reviewtext()
                    # 텍스트 -> 리스트화, 데이터 셋으로 만들기
        except:
            try:
                driver.find_element("xpath", '//*[@id="board_list"]/div/div[3]/div[3]/div/input').click() # 검색창 클릭
                time.sleep(1)
                driver.find_element("xpath",'//*[@id="board_list"]/div/div[3]/div[3]/div/input').send_keys(iwantreview[1]) # 리뷰 검색
                driver.find_element("xpath",'//*[@id="board_list"]/div/div[3]/div[3]/div/button').click()
                time.sleep(1)
                
                if driver.find_element("xpath", '//*[@id="board_list"]/div/div[2]/table/tbody/tr[3]/td/p') == True:# 만약 게시글이 없을 경우 패스
                    print('리뷰 게시물이 없습니다.')
                else:
                    driver.find_element("xpath",'//*[@id="board_list"]/div/div[2]/table/thead/tr/th[5]/a').click() # 후기 게시글이 있을 경우 추천수 내림차순
                    print('리뷰를 스크랩하고 있습니다.')
                    time.sleep(1)
                    reviewtext()
                
            except:
                pass

            # # else:
            #     driver.find_element("xpath",'//*[@id="board_list"]/div/div[2]/table/thead/tr/th[5]/a').click() # 후기 게시글이 있을 경우 추천수 내림차순
            #     time.sleep(1)
                
            #     for nums in range(1, 32):   # 후기 게시글 텍스트화하는 코드
            #         driver.find_element("xpath",'//*[@id="board_list"]/div/div[2]/table/tbody/tr[%d]/td[3]/div/a[1]' % (nums)).click()
            #         driver.find_element("xpath",'//*[@id="board_read"]/div/div[2]/div[2]/div[1]').text # 메인 글
            #         driver.find_element("xpath",'//*[@id="board_read"]/div/div[2]/div[1]/div[1]/div[2]/div[2]/div/p[5]').text # 추천, 조회수, 비추천
            #         driver.find_element("xpath",'//*[@id="board_read"]/div/div[2]/div[1]/div[1]/div[2]/div[2]/div/p[6]/span').text # 날짜
                    
            #         # '리뷰' 검색
            #     driver.find_element("xpath",'//*[@id="board_list"]/div/div[3]/div[3]/div/input').send_keys(iwantreview[1])
            #     driver.find_element("xpath",'//*[@id="board_list"]/div/div[3]/div[3]/div/button').click()
            #     time.sleep(1)
            #     # 리뷰 리스트 데이터 프레임에 집어넣는 코드

            #     try:
            #         driver.find_element("xpath", '//*[@id="board_list"]/div/div[2]/table/tbody/tr[3]/td/p')# 만약 게시글이 없을 경우 패스
            #         print('리뷰 게시물이 없습니다.')
            #     except:
            #         driver.get('https://bbs.ruliweb.com/game')
            #         time.sleep(2)
            #     # else:
            #         driver.find_element("xpath",'//*[@id="board_list"]/div/div[2]/table/thead/tr/th[5]/a').click() # 게시글이 있을 경우 추천수 내림차순

            # 대충 개별 후기 들어가서 스크랩하는 코드
            # 

    # else:
    #     driver.find_element("xpath", '//*[@id="game_search"]/form/ul/li/div/div[2]/div/p')



레인 월드의 게임 리뷰를 수집합니다.


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="game_search"]/form/ul/li/div/div[2]/div/p"}
  (Session info: chrome=105.0.5195.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00CFC0A3+2212003]
	Ordinal0 [0x00C92CC1+1780929]
	Ordinal0 [0x00BA465D+804445]
	Ordinal0 [0x00BD3475+996469]
	Ordinal0 [0x00BD363B+996923]
	Ordinal0 [0x00C01382+1184642]
	Ordinal0 [0x00BEEC64+1109092]
	Ordinal0 [0x00BFF5B2+1177010]
	Ordinal0 [0x00BEEA36+1108534]
	Ordinal0 [0x00BC83C9+951241]
	Ordinal0 [0x00BC9396+955286]
	GetHandleVerifier [0x00FA9CE2+2746722]
	GetHandleVerifier [0x00F9A234+2682548]
	GetHandleVerifier [0x00D8B34A+524234]
	GetHandleVerifier [0x00D89B60+518112]
	Ordinal0 [0x00C99FBC+1810364]
	Ordinal0 [0x00C9EA28+1829416]
	Ordinal0 [0x00C9EB15+1829653]
	Ordinal0 [0x00CA8744+1869636]
	BaseThreadInitThunk [0x768CFA29+25]
	RtlGetAppContainerNamedObjectPath [0x76F77A9E+286]
	RtlGetAppContainerNamedObjectPath [0x76F77A6E+238]
